# Pattern Recognition pipeline
The entire pipeline should be runnable from here.

### Genes data
**Raw genes data**
- Data is expected to be in *raw_data/genes/data.csv*
- Labels are expected to be in *raw_data/genes/labels.csv*

### Image data
**Raw image data**
- Animal images are expected to be in *raw_data/BigCats/[Animal]/*

### Feature selection
- Feature selection functions can be found in *feature_selection/[function].py*

### Classification
- Classification functions can be found in *classification/[function].py*

### Clustering
- Clustering functions can be found in *clustering/[function].py*

# Code


In [1]:
## Import ALL required imports

# Ignore SKlearn's deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Data imports
from raw_data.data_functions import load_num_data, load_img_data
from raw_data.data_load_MI_SVM import getImageData

# Feature selection imports
from feature_selection.pca import pca
from feature_selection.MI import MI_feature_select
from feature_selection.fourier_transform import ft_on_img_data
from feature_selection.edge_detection import edge_detection

# Classification imports
from classification.knn import knn, knn_gridsearch, knn_cross_validation
from classification.svm import svm_genes
from classification.decision_tree import decision_tree, cross_val_decision_tree
from classification.logistic_regression import logistic_regression, logistic_regression_cross_validation, logistic_regression_gridsearch

# Clustering imports
from clustering.kmeans import kmeans_train
# from clustering.fuzzy_c_means import fuzzy_c_means

# Misc imports
from sklearn.model_selection import train_test_split
from visualization.visualization import create_conf_matrix
from visualization.visualization import simple_pca_line_plot
from visualization.visualization import plot_tsne_num
from visualization.visualization import plot_tsne_img
from sklearn.manifold import TSNE

C:\Users\Boris\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Load Data

In [2]:
print("Loading num data...")
num_data, num_labels = load_num_data()
print("Loading img data...")
img_data, img_labels = load_img_data()
print("Loading img2 data...")
img_data_k, img_labels_k = getImageData("../Data-211216/Data/BigCats/" )  

Loading num data...
Loading img data...
Loading img2 data...
data load succ


## Visualize Data
**Create t-SNE embeddings**

In [12]:
num_data_embedded = TSNE(n_components=2, learning_rate='auto', init='random', random_state=0).fit_transform(num_data)
img_data_embedded = TSNE(n_components=2, learning_rate='auto', init='random', random_state=0).fit_transform(img_data)

**Plot the t-SNE embeddings**

In [15]:
# Genes data
# title = "t-SNE visualization of the Genes data"
# xlabel = "t-SNE dimension 1"
# ylabel = "t-SNE dimension 2"
# plot_tsne_num(num_data_embedded, num_labels, title, xlabel, ylabel)

# BigCats data
# title = "t-SNE visualization of the BigCats data"
# xlabel = "t-SNE dimension 1"
# ylabel = "t-SNE dimension 2"
# plot_tsne_img(img_data_embedded, img_labels, title, xlabel, ylabel)

## Feature Extraction

### PCA
**Apply PCA to the Genes data**

In [4]:
pca_num_data = pca(num_data.iloc[:,1:])

# Plot the cumulative variance explained per number of principal components
# simple_pca_line_plot(
#     pca_model.explained_variance_ratio_.cumsum()[:], 
#     title="Cum. variance explained / # principal components",
#     ylabel="Ratio of variance explained")

**Perform PCA experiment with k-NN**

In [11]:
# Result storage
pca_knn_res = []

# Perform cross-validated k-NN for each possible number of components
for num_components in range(1,802):
    selected_data = pca_num_data.iloc[:,:num_components]
    acc, f1 = knn_cross_validation(selected_data, num_labels[1], 3)
    pca_knn_res.append(acc)

# Plot the result
# simple_pca_line_plot(
#     pca_knn_res[:], 
#     title="Test accuracy per number of principal components",
#     ylabel="Test accuracy")

**Reduce PCA data to the most useful components (=79)**

In [4]:
pca_num_data_red = pca_num_data.iloc[:,:80]

NameError: name 'pca_num_data' is not defined

### MI

In [ ]:

mi_num_data=MI_feature_select(num_data,num_labels,thre=0.5)

feature selection start
Feature drop


/home/abe/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


478 features whose MI score over threshold are selected from 20531 features
feature selection end


### SIFT

### Fourier

In [3]:
ft_img_data = ft_on_img_data(img_data)

### Contour Images

In [6]:
edge_img_data = edge_detection(img_data)

C:\Users\Boris\Anaconda3\lib\site-packages\skimage\util\dtype.py:135: UserWarning: Possible precision loss when converting from int64 to float64
  .format(dtypeobj_in, dtypeobj_out))


## Classification

### k-NN - Genes - 80/20 split
**Raw data (best k = 15 [0.998090, 0.998437])**

In [32]:
# Create the 80/20 split
x_train, x_test, y_train, y_test = train_test_split(num_data, num_labels, test_size=0.2, random_state=42, stratify=num_labels)

# Tune k using gridsearch on the train set
# knn_gridsearch(x_train, y_train[1], [k for k in range(1,21,2)])

# Train using the optimal k (=5) and test on the test set
acc, f1, pred_labels = knn(x_train, y_train[1], x_test, y_test[1], k=15)
print(f'Accuracy = {acc} - F1-score = {f1}')

# Show a confusion matrix
# create_conf_matrix(y_test[1], pred_labels,  title="k-NN predictions using raw data")

Accuracy = 0.9937888198757764 - F1-score = 0.993758020635491


**PCA (best k = 13 [0.998785, 0.998437])**

In [35]:
# Create the 80/20 split
x_train, x_test, y_train, y_test = train_test_split(pca_num_data_red, num_labels, test_size=0.2, random_state=42, stratify=num_labels)

# Tune k using gridsearch on the train set
# knn_gridsearch(x_train, y_train[1], [k for k in range(1,21,2)])

# Train using the optimal k (=k) and test on the test set
acc, f1, pred_labels = knn(x_train, y_train[1], x_test, y_test[1], k=13)
print(f'Accuracy = {acc} - F1-score = {f1}')

# Show a confusion matrix
# create_conf_matrix(y_test[1], pred_labels,  title="k-NN predictions using PCA data")

Accuracy = 0.9813664596273292 - F1-score = 0.9811987061061624


**Mutual Information (best k = 5 [0.998437, 0.998437])**

In [34]:
# Create the 80/20 split
x_train, x_test, y_train, y_test = train_test_split(mi_num_data, num_labels, test_size=0.2, random_state=42, stratify=num_labels)

# Tune k using gridsearch on the train set
# knn_gridsearch(x_train, y_train[1], [k for k in range(1,21,2)])

# Train using the optimal k (=k) and test on the test set
acc, f1, pred_labels = knn(x_train, y_train[1], x_test, y_test[1], k=5)
print(f'Accuracy = {acc} - F1-score = {f1}')

# Show a confusion matrix
# create_conf_matrix(y_test[1], pred_labels, title="k-NN predictions using MI data")

Accuracy = 0.9937888198757764 - F1-score = 0.993758020635491


### k-NN - Genes - Cross Validation
**Raw data (best k = 3,5,7,9 [0.998752, 0.998750])**

In [35]:
# Tune k using gridsearch on the entire data set
# knn_gridsearch(num_data, num_labels[1], [k for k in range(1,21,2)])

# Evaluate with the optimal k (=k) using cross validation
acc, f1 = knn_cross_validation(num_data, num_labels[1], k=3)
print(f'Accuracy = {acc} - F1-score = {f1}')

Accuracy = 0.99875 - F1-score = 0.9987370977534912


**PCA (best k = 3 [0.997503, 0.997500])**

In [37]:
# Tune k using gridsearch on the entire data set
# knn_gridsearch(pca_num_data_red, num_labels[1], [k for k in range(1,21,2)])

# Evaluate with the optimal k (=k) using cross validation
acc, f1 = knn_cross_validation(pca_num_data_red, num_labels[1], k=3)
print(f'Accuracy = {acc} - F1-score = {f1}')

Accuracy = 0.9974999999999999 - F1-score = 0.9974441687344914


**Mutual Information (best k = 5-19 [0.997503, 0.997500])**

In [37]:
# Tune k using gridsearch on the entire data set
# knn_gridsearch(mi_num_data, num_labels[1], [k for k in range(1,21,2)])

# Evaluate with the optimal k (=k) using cross validation
acc, f1 = knn_cross_validation(mi_num_data, num_labels[1], k=5)
print(f'Accuracy = {acc} - F1-score = {f1}')

Accuracy = 0.9974999999999999 - F1-score = 0.9974441687344914


### k-NN - BigCats - 80/20 split
**Raw data (best k = 3 [0.520405, 0.235714])**

In [38]:
# Create the 80/20 split
x_train, x_test, y_train, y_test = train_test_split(img_data, img_labels, test_size=0.2, random_state=42, stratify=img_labels)

# Tune k using gridsearch on the train set
# knn_gridsearch(x_train, y_train, [k for k in range(1,21,2)])

# Train using the optimal k (=k) and test on the test set
acc, f1, pred_labels = knn(x_train, y_train, x_test, y_test, k=3)
print(f'Accuracy = {acc} - F1-score = {f1}')

# Show a confusion matrix
# create_conf_matrix(y_test, pred_labels, title="k-NN predictions using raw data")

Accuracy = 0.17647058823529413 - F1-score = 0.16657329598506068


**SIFT (best k = k)**

In [ ]:
# # Create the 80/20 split
# x_train, x_test, y_train, y_test = train_test_split(ft_img_data, img_labels, test_size=0.2, random_state=42, stratify=img_labels)

# # Tune k using gridsearch on the train set
# # knn_gridsearch(x_train, y_train, [k for k in range(1,21,2)])

# # Train using the optimal k (=5) and test on the test set
# acc, f1, pred_labels = knn(x_train, y_train, x_test, y_test, k=17)

# # Show a confusion matrix
# create_conf_matrix(y_test, pred_labels, title="k-NN predictions using SIFT data")

**Fourier (best k = 17 [0.361922, 0.300549])**

In [39]:
# Create the 80/20 split
x_train, x_test, y_train, y_test = train_test_split(ft_img_data, img_labels, test_size=0.2, random_state=42, stratify=img_labels)

# Tune k using gridsearch on the train set
# knn_gridsearch(x_train, y_train, [k for k in range(1,21,2)])

# Train using the optimal k (=5) and test on the test set
acc, f1, pred_labels = knn(x_train, y_train, x_test, y_test, k=17)
print(f'Accuracy = {acc} - F1-score = {f1}')

# Show a confusion matrix
# create_conf_matrix(y_test, pred_labels, title="k-NN predictions using FT data")

Accuracy = 0.4411764705882353 - F1-score = 0.3927932576539388


### k-NN - BigCats - Cross Validation
**Raw data (best k = 7 [0.384967, 0.247059])**

In [40]:
# Tune k using gridsearch on the entire data set
# knn_gridsearch(img_data, img_labels, [k for k in range(1,21,2)])

# Evaluate with the optimal k (=k) using cross validation
acc, f1 = knn_cross_validation(img_data, img_labels, k=7)
print(f'Accuracy = {acc} - F1-score = {f1}')

Accuracy = 0.24705882352941178 - F1-score = 0.2068569992099404


**SIFT (best k = k)**

**Fourier (best k = 13 [0.425490, 0.382353])**

In [41]:
# Tune k using gridsearch on the entire data set
# knn_gridsearch(ft_img_data, img_labels, [k for k in range(1,21,2)])

# Evaluate with the optimal k (=k) using cross validation
acc, f1 = knn_cross_validation(ft_img_data, img_labels, k=13)
print(f'Accuracy = {acc} - F1-score = {f1}')

Accuracy = 0.38235294117647056 - F1-score = 0.33473941091588155


### Logistic Regression - BigCats - 80/20
**Raw data (best solver = lbfgs, best C = 1)**

In [14]:
# Create the 80/20 split
x_train, x_test, y_train, y_test = train_test_split(img_data, img_labels, test_size=0.2, random_state=42, stratify=img_labels)

# Tune k using gridsearch on the train set
# solvers = ["newton-cg", "lbfgs", "liblinear"]
# c_values = [0.1, 1, 10, 100]
# logistic_regression_gridsearch(x_train, y_train, solvers, c_values)

# Train using the optimal k (=k) and test on the test set
acc, f1, pred_labels = logistic_regression(x_train, y_train, x_test, y_test, solver="lbfgs", c_value=1)
print(f'Accuracy = {acc} - F1-score = {f1}')

# Show a confusion matrix
# create_conf_matrix(y_test, pred_labels, title="Logistic regression predictions using raw data")

Accuracy = 0.3235294117647059 - F1-score = 0.29150326797385623


**SIFT (best solver = x, best C = y)**

In [ ]:
# # Create the 80/20 split
# x_train, x_test, y_train, y_test = train_test_split(sift_img_data, img_labels, test_size=0.2, random_state=42, stratify=img_labels)

# # Tune k using gridsearch on the train set
# solvers = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
# c_values = [0.0001, 0.001, 0.1, 1, 10, 100, 1000, 10000]
# logistic_regression_gridsearch(x_train, y_train, solvers, c_values)

# # Train using the optimal k (=k) and test on the test set
# # acc, f1, pred_labels = logistic_regression(x_train, y_train, x_test, y_test, solver="newton-cg", c_value=10)
# # print(f'Accuracy = {acc} - F1-score = {f1}')

# # Show a confusion matrix
# # create_conf_matrix(y_test, pred_labels, title="Logistic regression predictions using raw data")

**Fourier (best solver = liblinear, best C = 0.1)**

In [4]:
# Create the 80/20 split
x_train, x_test, y_train, y_test = train_test_split(ft_img_data, img_labels, test_size=0.2, random_state=42, stratify=img_labels)

# Tune k using gridsearch on the train set
# solvers = ["newton-cg", "lbfgs", "liblinear"]
# c_values = [0.1, 1, 10, 100]
# logistic_regression_gridsearch(x_train, y_train, solvers, c_values)

# Train using the optimal k (=k) and test on the test set
acc, f1, pred_labels = logistic_regression(x_train, y_train, x_test, y_test, solver="liblinear", c_value=0.1)
print(f'Accuracy = {acc} - F1-score = {f1}')

# Show a confusion matrix
# create_conf_matrix(y_test, pred_labels, title="Logistic regression predictions using FT data")

Accuracy = 0.4411764705882353 - F1-score = 0.4169185460745563


### Logistic Regression - BigCats - Cross Validation
**Raw data  (best solver = liblinear, best C = 0.1)**

In [4]:
# Tune the solver and C parameters using gridsearch on the entire data set
# solvers = ["newton-cg", "lbfgs", "liblinear"]
# c_values = [0.1, 1, 10, 100]
# logistic_regression_gridsearch(img_data, img_labels, solvers, c_values)

# Evaluate with the optimal k (=k) using cross validation
acc, f1 = logistic_regression_cross_validation(img_data, img_labels, solver="liblinear", c_value=0.1)
print(f'Accuracy = {acc} - F1-score = {f1}')

Accuracy = 0.27647058823529413 - F1-score = 0.25509506833036244


**SIFT  (best solver = x, best C = y)**

In [ ]:
# # Tune the solver and C parameters using gridsearch on the entire data set
# solvers = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
# c_values = [0.1, 1, 10, 100]
# logistic_regression_gridsearch(sift_img_data, img_labels, solvers, c_values)

# # Evaluate with the optimal k (=k) using cross validation
# acc, f1 = logistic_regression_cross_validation(sift_img_data, img_labels, solver="", c_value=10)
# print(f'Accuracy = {acc} - F1-score = {f1}')

**Fourier (best solver = linear, best C = 0.1)**

In [5]:
# Tune the solver and C parameters using gridsearch on the entire data set
# solvers = ["newton-cg", "lbfgs", "liblinear"]
# c_values = [0.1, 1, 10, 100]
# logistic_regression_gridsearch(ft_img_data, img_labels, solvers, c_values)

# Evaluate with the optimal k (=k) using cross validation
acc, f1 = logistic_regression_cross_validation(ft_img_data, img_labels, solver="liblinear", c_value=0.1)
print(f'Accuracy = {acc} - F1-score = {f1}')

Accuracy = 0.39999999999999997 - F1-score = 0.35637000254647316


### Naive Bayes

#### 80/20 split

#### K-fold

### Decision Trees

#### Parameter Sweep

In [8]:
print("Performing parameter sweep...")
accuracies = []
depths = [3,4,5,6,7,8,9,10]
for depth in depths:
    print(f"Trying max depth of {depth}...", end='\r')
    accuracies.append(decision_tree(num_data, num_labels)[0])
print(accuracies)
best_depth = depths[accuracies.index(max(accuracies))]
print(f"test accuracy: {max(accuracies)} for depth {depths[accuracies.index(max(accuracies))]}")

Performing parameter sweep...
[0.9813664596273292, 0.9627329192546584, 0.9751552795031055, 0.968944099378882, 0.9875776397515528, 0.9875776397515528, 0.9565217391304348, 0.968944099378882]
test accuracy: 0.9875776397515528 for depth 7


#### 80/20 split

In [10]:
# Raw data
print("using decision tree classifier on genes raw")
accuracy, f1_score, y_pred = decision_tree(num_data, num_labels, depth=best_depth)
print(f"test accuracy & f1: {accuracy, f1_score}")
#create_conf_matrix(num_labels, y_pred)

# PCA data
print("using decision tree classifier on genes PCA")
accuracy, f1_score, y_pred = decision_tree(pca_num_data, num_labels, depth=best_depth)
print(f"test accuracy & f1: {accuracy, f1_score}")
#create_conf_matrix(num_labels, y_pred)

# MI data
print("using decision tree classifier on genes MI")
accuracy, f1_score, y_pred = decision_tree(mi_num_data, num_labels, depth=best_depth)
print(f"test accuracy & f1: {accuracy, f1_score}")
#create_conf_matrix(num_labels, y_pred)



using decision tree classifier on genes raw
test accuracy & f1: (0.9813664596273292, 0.981386992454186)
using decision tree classifier on genes PCA
test accuracy & f1: (0.9006211180124224, 0.898379039885027)
using decision tree classifier on genes MI
test accuracy & f1: (0.9751552795031055, 0.9753293590801105)


#### K-fold

In [14]:
k = 10
# Raw data
print(f"using decision tree classifier on genes raw with {k}-fold")
accuracy, f1 = cross_val_decision_tree(num_data, num_labels, cv=k, depth=best_depth)
print("%0.3f test accuracy with a standard deviation of %0.3f" % (accuracy.mean(), accuracy.std()))
print("%0.3f test f1 with a standard deviation of %0.3f" % (f1.mean(), f1.std()))

# PCA data
print(f"using decision tree classifier on genes PCA with {k}-fold")
accuracy, f1 = cross_val_decision_tree(pca_num_data, num_labels, cv=k, depth=best_depth)
print("%0.3f test accuracy with a standard deviation of %0.3f" % (accuracy.mean(), accuracy.std()))
print("%0.3f test f1 with a standard deviation of %0.3f" % (f1.mean(), f1.std()))

# MI data
print(f"using decision tree classifier on genes MI with {k}-fold")
accuracy, f1 = cross_val_decision_tree(mi_num_data, num_labels, cv=k, depth=best_depth)
print("%0.3f test accuracy with a standard deviation of %0.3f" % (accuracy.mean(), accuracy.std()))
print("%0.3f test f1 with a standard deviation of %0.3f" % (f1.mean(), f1.std()))

using decision tree classifier on genes raw with 10-fold
0.980 test accuracy with a standard deviation of 0.000
0.980 test f1 with a standard deviation of 0.000
using decision tree classifier on genes PCA with 10-fold
0.916 test accuracy with a standard deviation of 0.000
0.916 test f1 with a standard deviation of 0.000
using decision tree classifier on genes MI with 10-fold
0.984 test accuracy with a standard deviation of 0.000
0.984 test f1 with a standard deviation of 0.000


### Logistic Regression

#### 80/20 split

#### K-fold

### SVM

#### 80/20 split

In [7]:
svm_genes(mi_num_data,num_labels)

svm train start
new model saved
accuracy on the training subset:1.000
accuracy on the test subset:0.994


/home/abe/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### K-fold

## Clustering

### K-Means

In [3]:
kmeans_train(img_data_k, img_labels_k)

score: -680557997.8520824
MI_evaluate_score_of_test: 0.22804604671270068
----------------k= 0 ----------------
['Tiger' 'Cheetah' 'Tiger' 'Tiger' 'Tiger' 'Lion' 'Tiger' 'Cheetah'
 'Jaguar' 'Cheetah' 'Cheetah' 'Lion' 'Tiger' 'Tiger' 'Leopard' 'Lion']
----------------k= 1 ----------------
['Jaguar' 'Jaguar' 'Jaguar' 'Tiger' 'Cheetah' 'Cheetah' 'Tiger' 'Leopard'
 'Cheetah' 'Cheetah' 'Jaguar' 'Lion' 'Lion' 'Cheetah' 'Jaguar' 'Leopard'
 'Lion' 'Leopard' 'Jaguar' 'Jaguar' 'Cheetah' 'Lion' 'Tiger' 'Jaguar'
 'Leopard']
----------------k= 2 ----------------
['Leopard' 'Lion' 'Tiger' 'Tiger' 'Tiger' 'Lion' 'Cheetah' 'Leopard'
 'Lion' 'Lion' 'Jaguar']
----------------k= 3 ----------------
['Lion' 'Tiger' 'Lion' 'Tiger' 'Cheetah' 'Tiger' 'Leopard' 'Jaguar'
 'Leopard' 'Tiger' 'Cheetah' 'Lion' 'Jaguar' 'Lion' 'Cheetah' 'Cheetah'
 'Leopard' 'Lion' 'Leopard' 'Lion' 'Leopard' 'Lion' 'Leopard' 'Tiger'
 'Tiger']
----------------k= 4 ----------------
['Lion' 'Jaguar' 'Cheetah' 'Tiger' 'Lion' 'Cheetah' 'Ch

### Fuzzy C-Means

In [10]:
print("RAW NUM")
fuzzy_c_means(num_data, 3)
print("PCA NUM")
fuzzy_c_means(pca_num_data, 3)
print("MI NUM")
fuzzy_c_means(mi_num_data, 3)

print("RAW IMG")
fuzzy_c_means(img_data, 3)
print("FT IMG")
fuzzy_c_means(ft_img_data, 3)



RAW NUM
fcm.partition_coefficient=0.040000000000048164
fcm.partition_entropy_coefficient=0.46438561897729874
PCA NUM
fcm.partition_coefficient=0.040000000000009125
fcm.partition_entropy_coefficient=0.4643856189774396
MI NUM
fcm.partition_coefficient=0.040000000000133616
fcm.partition_entropy_coefficient=0.46438561897699054
RAW IMG
fcm.partition_coefficient=0.04000000000005647
fcm.partition_entropy_coefficient=0.4643856189772687
FT IMG
fcm.partition_coefficient=0.040000000000025245
fcm.partition_entropy_coefficient=0.46438561897738134


-----------------------------------------------------
*end*